## FDSN Waveform Ingestion and Processing

Downloads I06AU and I52GB waveforms from IRIS FSDN services using Obpsy Mass Downloader and then converts them the .SAC file with it's stats for further processing. The .SAC files will be then further processed using "Bartlett" beamforming method and Adaptive F-Detector to identify signals most related to Anak Krakatau Volcanic Activity. These specific volcanic activity signals will be later ingested into the algorithm for the Deep Learning processing. This notebook will do as follows:\

This notebook consists of 3 parts:
1. Dowloading Data using Obspy Mass Downloader
2. Conversion of MSEED Files to .SAC
3. Implementing FK Beamforming to each station group
4. Running Adaptive F-Detector to Isolate relevant Waveform
5. Calculating Best Beam using Laslo's Method


In [2]:
import sys
sys.path.append('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/')

import obspy
from obspy import UTCDateTime, read, read_inventory
from obspy.clients.fdsn.mass_downloader import GlobalDomain, \
    Restrictions, MassDownloader
from obspy.core.util.attribdict import AttribDict
import os
import subprocess
import glob
from tqdm.notebook import tqdm
from tqdm import tqdm
from src.utils.converter import *
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

### 1. Downloading Data Using Obspy Mass Downloader

Download the data ranging from 2018-06-24T00:00:00 until 2019-09-03T00:00:00

In [ ]:
# 1.1 I06AU Waveform Download

domain = GlobalDomain()

restrictions = Restrictions(
    # Get data for a whole year.
    starttime=obspy.UTCDateTime(2018, 6, 24),
    endtime=obspy.UTCDateTime(2019, 9, 3),
    # Chunk it to have one file per day.
    chunklength_in_sec=86400,
    network="IM", station="I06H*", location="", channel="BDF",
    # The typical use case for such a data set are noise correlations where
    # gaps are dealt with at a later stage.
    reject_channels_with_gaps=False,
    # Same is true with the minimum length. All data might be useful.
    minimum_length=0.0,
    # Guard against the same station having different names.
    minimum_interstation_distance_in_m=100.0)

mdl = MassDownloader(providers=["IRIS"])
mdl.download(domain, restrictions, mseed_storage="waveform_collection/I06AU/WAVEFORM_I06AU_MSEED",
             stationxml_storage="waveform_collection/I06AU/I06AU_STATIONS")

In [ ]:
# 1.2 I52GB Waveform Download

domain = GlobalDomain()

restrictions = Restrictions(
    starttime=obspy.UTCDateTime(2018, 6, 24),
    endtime=obspy.UTCDateTime(2019, 9, 3),
    chunklength_in_sec=86400,
    network="IM", station="I52H*", location="", channel="BDF",
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0)


mdl = MassDownloader(providers=["IRIS"])
mdl.download(domain, restrictions, mseed_storage="/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_MSEED",
             stationxml_storage="/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_STATIONS")

### 2. Conversion of MSEED Files to .SAC
Converting MSEED Files to SAC Complete with Important Headers

In [ ]:
# 2.1 I06AU Waveform Conversion

input_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_MSEED'
output_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_SAC'
stationxml_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_STATIONS'

# Run the Function
mseed_to_sac(input_directory, output_directory, stationxml_directory)

In [ ]:
# 2.2 I52GB Waveform Conversion

input_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_MSEED'
output_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_SAC'
stationxml_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_STATIONS'

# Run the Function
mseed_to_sac(input_directory, output_directory, stationxml_directory)


### 3. Bartlett FK Beamforming

In [ ]:
# 4.1 Running Beamforming on I06AU Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --freq-min 0.7 --freq-max 4 --back-az-min 50 --back-az-max 60 --window-len 600 --sub-window-len 30 --window-step 300"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

In [ ]:
# 4.1 Running Beamforming on I52GB Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I52GB/WAVEFORM_I52GB_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

### 4. Adaptive F-Detector

In [ ]:
# 5.2 Running AFD for I06AU Station

folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FK_RESULTS'

# List all files in the folder
files = os.listdir(folder_path)
full_paths = [os.path.join(folder_path, file) for file in files]

# Using ThreadPoolExecutor to process files concurrently
with ThreadPoolExecutor(8) as executor:
    # Map the executor to the process_file function and the list of file paths
    results = list(tqdm(executor.map(lambda file_path: subprocess.run(f"infrapy run_fd --local-fk-label {file_path} --p-value 0.05 --back-az-width 5 --min-duration 25 --merge-dets 'False'", shell=True), full_paths), total=len(full_paths), desc="Processing files"))


In [ ]:
# Path to the folder containing the files
folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I52GB/I52GB_FK'

# List all files in the folder
files = os.listdir(folder_path)
full_paths = [os.path.join(folder_path, file) for file in files]

# Using ThreadPoolExecutor to process files concurrently
with ThreadPoolExecutor(8) as executor:
    # Map the executor to the process_file function and the list of file paths
    results = list(tqdm(executor.map(lambda file_path: subprocess.run(f"infrapy run_fd --local-fk-label {file_path} --p-value 0.05 --back-az-width 5", shell=True), full_paths), total=len(full_paths), desc="Processing files"))


### 5. Laslo's Best Beam Calculation

In [ ]:
import subprocess
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from tqdm.notebook import tqdm

# Load the CSV file
file_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/notebook/Best-Beam_ITeration/BEST-BEAM_DATABASE_Unique_6.csv'  # Replace with your actual CSV file path
df = pd.read_csv(file_path)

# Function to generate CLI command for each row
def generate_command(row):
    return (
        f"infrapy utils best-beam --local-wvfrms '{row['SACFile']}' "
        f"--local-fk-label '{row['FKResults']}' --signal-start '{row['Start']}' "
        f"--signal-end '{row['End']}' --trace-vel {row['Trace Velocity']} "
        f"--back-az {row['Back Azimuth']} --freq-min {row['Freq Min']} "
        f"--freq-max {row['Freq Max']} --hold-figure False"
    )

# Generating CLI commands for each row
cli_commands = df.apply(generate_command, axis=1).tolist()

# Function to execute a command
def run_command(cmd):
    try:
        subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return "Success"
    except subprocess.CalledProcessError as e:
        return f"Failed: {e}"

# Running the commands concurrently and monitoring with tqdm
with ThreadPoolExecutor(max_workers=12) as executor:
    results = list(tqdm(executor.map(run_command, cli_commands), total=len(cli_commands)))


In [ ]:
# Paths to the folders containing the CSV files and output files
csv_folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/notebook/Best-Beam_ITeration'
output_folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FK_RESULTS_300/'

def run_command_and_rename(cmd, csv_index, fk_label):
    try:
        # Execute the command
        subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Extract the timestamp part from the FKResults filename
        timestamp_part = fk_label.split('/')[-1].replace('.fk_results.dat', '')

        # Original and new output filename formats
        original_output_file = os.path.join(output_folder_path, timestamp_part + ".best-beam.dat")
        new_output_filename = os.path.join(output_folder_path, f"{csv_index}_{timestamp_part}.best-beam.dat")

        # Rename the file
        if os.path.exists(original_output_file):
            os.rename(original_output_file, new_output_filename)
        return "Success"
    except subprocess.CalledProcessError as e:
        return f"Failed: {e}"

csv_files = glob.glob(os.path.join(csv_folder_path, '*.csv'))
for csv_index, file_path in enumerate(csv_files, start=1):
    df = pd.read_csv(file_path)
    cli_commands = df.apply(generate_command, axis=1).tolist()

    with ThreadPoolExecutor(max_workers=12) as executor:
        futures = [executor.submit(run_command_and_rename, cmd, csv_index, row['FKResults']) for cmd, row in zip(cli_commands, df.to_dict('records'))]
        results = list(tqdm((future.result() for future in futures), total=len(cli_commands)))

    # Process results or perform additional actions as needed
